# TP - EnKF - Ensemble Kalman Filter

Nous allons explorer l'assimilation de données sur un système chaotique à partir du système de [Lorenz 1963](https://doi.org/10.1177/0309133315623099
). Il s'agit d'un exemple historique de la représentation des systèmes chaotiques en météorologie.

[Edward Lorenz](https://fr.wikipedia.org/wiki/Edward_Lorenz) est connu pour sa conférence : _Le battement d'ailes d'un papillon au Brésil peut-il déclencher une tornade au Texas ?_

Il apporte un regard pionnier sur l'étude de la fiabilité des systèmes de prévision météo au regard de la théorie du chaos.

L'état du système est constitué par $X = [a, b, c] \in \mathbb{R}^3$.

$$\partial_t a = \sigma (b-a)$$
$$\partial_t b = a(\rho -c) - b$$
$$\partial_t c = ab - \beta c$$

avec $\sigma = 10, \rho = 28, \beta = \frac{8}{3}$

On note $\tilde{a}, \tilde{b}$ sont les variables centrées réduites respectivement associées à $a, b$.

On considère un système discret de la forme :

$$x_{i+1} = \mathcal{M}(x_i) + \eta$$

Où $\mathcal{M}$ est l'intégration du système de Lorenz, et $\eta$ un bruit brownien.

1. Implémenter l'intégration numérique du système, avec une méthode d'Euler Explicite

In [ ]:
def forward_euler(x, dt):
  X = np.zeros((3, 1))
  X[0] = x[0] + dt * (sigma * (x[1] - x[0]))
  X[1] = x[1] + dt * (x[0] * (rho - x[2]) - x[1])
  X[2] = x[2] + dt * (x[0] * x[1] - beta * x[2])

X = np.array([1, 1, 1])
dt = 0.01

sigma = 10
rho = 28
beta = 8/3